# GPS数据清洗

In [4]:
# Import relevant libraries
import pandas as pd # Pandas (pd) offers data structures and operations for manipulating numerical data
import numpy as np # Numpy(np) supports operations on large multi-dimensional arrays and matrices
import matplotlib.pyplot as plt #Matplotlib supports plotting data in Python
import seaborn as sns #Seaborn is a Python data visualization library based on matplotlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # Machine learning library for the Python programming language
import pickle
import os
from tqdm import tqdm
import sys 
sys.path.append('../')

## taxi

In [4]:
#获取taxi文件夹下所有csv文件
path = '../Datasets/GPS_data/taxi'
file_list = []
for root,dirs,files in os.walk(path): #根目录，文件夹，文件
        for file in files:
                file_list.append(os.path.join(root,file))
print(file_list, '\n', len(file_list))

['../Datasets/GPS_data/taxi/GPS_taxi_1.0.pkl', '../Datasets/GPS_data/taxi/GPS_taxi.pkl', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00011', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00043', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00026', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00044', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00048', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00008', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00025', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00023', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00027', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00034', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00000', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00012', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00047', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00018', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00001', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00020', '../Datasets/GPS_data/taxi/2018-10-01/part-r-00002', '../Datase

In [ ]:
#多个csv文件合并为一个pickle文件
dataframes = []
for file in tqdm(file_list):
    df = pd.read_csv(file, sep=",", header=None)
    dataframes.append(df)
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.to_pickle(os.path.join(path,"GPS_taxi.pkl"))
print(combined_df.shape)

In [2]:
path = '../Datasets/GPS_data/taxi'
with open(os.path.join(path,"GPS_taxi.pkl"), 'rb') as f:
    data_p = pickle.load(f)
data_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131897448 entries, 0 to 131897447
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         object 
 1   lon        float64
 2   lat        float64
 3   time       object 
 4   speed      int64  
 5   direction  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 5.9+ GB


In [10]:
#添加列名
header = ['id','lon','lat','time','device','speed','direction','pstatus','warning','cardno','status','color']
data_p.columns = header

In [30]:
#数据清洗
data_p = data_p.drop(['device', 'pstatus', 'warning', 'cardno', 'status', 'color'], axis=1) #删除不需要的列

In [ ]:
#数据清洗
#删除异常值

In [4]:
a = pd.DataFrame(data_p['direction'].value_counts())
#a = data_p['direction'].value_counts()

a

,count
direction,
0,14211817
179,1498618
134,1081560
89,1060628
44,1058942
...,...
426,3775
420,3643
402,3144


In [5]:
data_p

,id,lon,lat,time,speed,direction
0,023832,113.961416,22.553134,2018-10-01T21:04:00.000Z,0,0
1,023832,113.961416,22.553134,2018-10-01T23:11:08.000Z,0,0
2,023832,113.961416,22.553134,2018-10-01T23:12:37.000Z,0,0
3,023832,113.961416,22.553134,2018-10-01T23:43:47.000Z,0,0
4,225653,114.229512,22.745734,2018-10-01T01:07:12.000Z,0,220
...,...,...,...,...,...,...
131897443,粤BB995学,113.889640,22.591332,2019-10-01T15:15:58.000Z,0,244
131897444,粤BB995学,113.891265,22.593610,2019-10-01T18:26:12.000Z,0,264
131897445,粤BB995学,113.892233,22.591445,2019-10-01T18:25:22.000Z,0,338
131897446,粤BB995学,113.892523,22.590298,2019-10-01T18:24:57.000Z,0,68


In [16]:
fsize = os.path.getsize(os.path.join(path,'GPS_taxi_1.0.pkl')) #获取文件大小,单位为byte
fsize = fsize/float(1024*1024*1024) #单位变成G
fsize

8.297303197905421

In [32]:
#保存文件
with open(os.path.join(path,"GPS_taxi.pkl"), 'wb') as f:
    pickle.dump(data_p, f)

In [32]:
#坐标转换
#将taxi的GPS数据由wgs84坐标转换为gcj02坐标下的web墨卡托坐标
from coordinate_transform import wgs_to_gcj02_to_mercator

#读取文件
with open("../Datasets/GPS_data/taxi/GPS_taxi.pkl", 'rb') as f:
    data = pickle.load(f, encoding='bytes')

#data = pd.read_csv("../Datasets/GPS_data/taxi/2018-10-01/part-r-00000", sep=",", header=None)
#data.columns = ['id','lon','lat','time','device','speed','direction','pstatus','warning','cardno','status','color']

transformed_coords = []
for lon, lat in zip(data['lon'].values, data['lat'].values):
    transformed_coords.append(wgs_to_gcj02_to_mercator(lon, lat))
    
data['lon'], data['lat'] = zip(*transformed_coords)
data

,id,lon,lat,time,speed,direction
0,023832,1.268667e+07,2.577708e+06,2018-10-01T21:04:00.000Z,0,0
1,023832,1.268667e+07,2.577708e+06,2018-10-01T23:11:08.000Z,0,0
2,023832,1.268667e+07,2.577708e+06,2018-10-01T23:12:37.000Z,0,0
3,023832,1.268667e+07,2.577708e+06,2018-10-01T23:43:47.000Z,0,0
4,225653,1.271651e+07,2.600958e+06,2018-10-01T01:07:12.000Z,0,220
...,...,...,...,...,...,...
131897443,粤BB995学,1.267868e+07,2.582304e+06,2019-10-01T15:15:58.000Z,0,244
131897444,粤BB995学,1.267886e+07,2.582579e+06,2019-10-01T18:26:12.000Z,0,264
131897445,粤BB995学,1.267897e+07,2.582318e+06,2019-10-01T18:25:22.000Z,0,338
131897446,粤BB995学,1.267900e+07,2.582179e+06,2019-10-01T18:24:57.000Z,0,68


In [33]:
#保存文件
with open("../Datasets/GPS_data/taxi/GPS_taxi_2.0.pkl", 'wb') as f:
    pickle.dump(data, f)

In [2]:
#test
#将taxi的GPS数据由wgs84坐标转换为gcj02坐标下的web墨卡托坐标
from coordinate_transform import wgs_to_gcj02_to_mercator

#读取文件

data = pd.read_csv("../Datasets/GPS_data/taxi/2018-10-01/part-r-00000", sep=",", header=None)
data.columns = ['id','lon','lat','time','device','speed','direction','pstatus','warning','cardno','status','color']

transformed_coords = []
for lon, lat in zip(data['lon'].values, data['lat'].values):
    transformed_coords.append(wgs_to_gcj02_to_mercator(lon, lat))
    
data['lon'], data['lat'] = zip(*transformed_coords)
data

,id,lon,lat,time,device,speed,direction,pstatus,warning,cardno,status,color
0,000284,1.269476e+07,2.574213e+06,2018-10-01T02:14:03.000Z,000284,0,0,0,0,000284,0,黄色
1,000284,1.269476e+07,2.574213e+06,2018-10-01T02:25:59.000Z,000284,0,0,0,0,000284,0,黄色
2,023832,1.268663e+07,2.577679e+06,2018-10-01T02:26:40.000Z,023832,0,0,0,0,023832,0,黄色
3,023832,1.268663e+07,2.577679e+06,2018-10-01T02:59:15.000Z,023832,0,0,0,0,023832,0,黄色
4,023832,1.268663e+07,2.577679e+06,2018-10-01T06:01:24.000Z,023832,0,0,0,0,023832,0,黄色
...,...,...,...,...,...,...,...,...,...,...,...,...
648866,粤BFJ099监控2,1.264902e+07,2.633668e+06,2018-10-01T11:21:44.000Z,粤BFJ099监控2,0,0,0,0,粤BFJ099监控2,0,黄色
648867,粤BFJ099监控2,1.264902e+07,2.633668e+06,2018-10-01T12:04:20.000Z,粤BFJ099监控2,0,0,0,0,粤BFJ099监控2,0,黄色
648868,粤BFJ099监控2,1.264902e+07,2.633668e+06,2018-10-01T14:52:42.000Z,粤BFJ099监控2,0,0,0,0,粤BFJ099监控2,0,黄色
648869,粤BFJ099监控2,1.264902e+07,2.633668e+06,2018-10-01T21:11:47.000Z,粤BFJ099监控2,0,0,0,0,粤BFJ099监控2,0,黄色


In [5]:
range = [(12694700, 2574220), (12694800, 2574213)]
type(range)

list

In [6]:
#selected_rows = data[((data['lon'] >= 12694700) & (data['lon'] <= 12694800)) & (data['lat'] >= 2574213 & (data['lat'] <= 2574220))]
selected_rows = data[(data['lon'].between(range[0][0], range[1][0])) & (data['lat'].between(range[1][1], range[0][1]))]
selected_rows

,id,lon,lat,time,device,speed,direction,pstatus,warning,cardno,status,color
0,000284,1.269476e+07,2.574213e+06,2018-10-01T02:14:03.000Z,000284,0,0,0,0,000284,0,黄色
1,000284,1.269476e+07,2.574213e+06,2018-10-01T02:25:59.000Z,000284,0,0,0,0,000284,0,黄色
194739,粤BR44Y2,1.269477e+07,2.574215e+06,2018-10-01T04:13:20.000Z,粤BR44Y2,0,45,0,0,粤BR44Y2,0,蓝色
194740,粤BR44Y2,1.269477e+07,2.574215e+06,2018-10-01T05:16:26.000Z,粤BR44Y2,0,45,0,0,粤BR44Y2,0,蓝色
194741,粤BR44Y2,1.269477e+07,2.574215e+06,2018-10-01T05:25:39.000Z,粤BR44Y2,0,45,0,0,粤BR44Y2,0,蓝色
194742,粤BR44Y2,1.269477e+07,2.574215e+06,2018-10-01T05:56:11.000Z,粤BR44Y2,0,45,0,0,粤BR44Y2,0,蓝色
194743,粤BR44Y2,1.269477e+07,2.574215e+06,2018-10-01T06:41:50.000Z,粤BR44Y2,0,45,0,0,粤BR44Y2,0,蓝色
194744,粤BR44Y2,1.269477e+07,2.574215e+06,2018-10-01T06:43:54.000Z,粤BR44Y2,0,45,0,0,粤BR44Y2,0,蓝色
194745,粤BR44Y2,1.269477e+07,2.574215e+06,2018-10-01T07:09:17.000Z,粤BR44Y2,0,45,0,0,粤BR44Y2,0,蓝色


# New taxi

In [5]:
#打开文件
file = "../Datasets/taxi_gps/2023-03-01"
df = pd.read_csv(file, sep=",", header=None)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220675391 entries, 0 to 220675390
Data columns (total 17 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       object 
 1   1       object 
 2   2       float64
 3   3       float64
 4   4       object 
 5   5       float64
 6   6       float64
 7   7       float64
 8   8       int64  
 9   9       int64  
 10  10      float64
 11  11      float64
 12  12      float64
 13  13      float64
 14  14      float64
 15  15      float64
 16  16      int64  
dtypes: float64(11), int64(3), object(3)
memory usage: 28.0+ GB


In [6]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2023-03-01 00:00:00,粤BAV0409,114.064263,22.518955,2023-02-28 22:19:07,NaN,5.0,96.0,256,0,NaN,NaN,NaN,NaN,119166.2,0.0,0
1,2023-03-01 00:00:00,粤BDR4746,114.137750,22.570602,2023-02-28 22:03:05,NaN,28.0,148.0,768,268566528,NaN,NaN,NaN,NaN,417048.0,0.0,45
2,2023-03-01 00:00:01,粤BD97957,114.129683,22.551567,2023-02-28 22:15:28,NaN,0.0,28.0,768,268435456,NaN,NaN,NaN,NaN,674710.0,0.0,81
3,2023-03-01 00:00:01,粤BDF7305,114.142747,22.555637,2023-02-28 22:23:36,NaN,53.0,94.0,256,268435456,NaN,NaN,NaN,NaN,602294.0,0.0,70
4,2023-03-01 00:00:01,粤BAN1395,114.120495,22.547132,2023-02-28 22:19:39,NaN,55.0,268.0,768,131072,NaN,NaN,NaN,NaN,95.1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220675386,2023-03-01 23:59:57,粤BDK1409,114.269750,22.768173,2023-03-01 22:26:15,NaN,34.0,232.0,256,131072,NaN,NaN,NaN,NaN,116020.1,0.0,0
220675387,2023-03-01 23:59:57,粤BD87070,114.020517,22.531595,2023-03-01 22:39:13,NaN,4.0,144.0,256,268435456,NaN,NaN,NaN,NaN,553679.0,0.0,58
220675388,2023-03-01 23:59:57,粤BDG9356,114.122922,22.541225,2023-03-01 22:33:45,NaN,12.0,304.0,768,268435456,NaN,NaN,NaN,NaN,556536.0,0.0,84
220675389,2023-03-01 23:59:57,粤BD04268,113.811762,22.627985,2023-03-01 22:19:26,NaN,0.0,296.0,256,268435456,NaN,NaN,NaN,NaN,663896.0,0.0,65


In [9]:
#添加列名
header = ['time','id','lon','lat','ptime','no_1','speed','no_2','no_7','cardno','no_3','no_4','no_5','no_6','mileage','status','direction']

df.columns = header

In [10]:
df

,time,id,lon,lat,ptime,no_1,speed,no_2,no_7,cardno,no_3,no_4,no_5,no_6,mileage,status,direction
0,2023-03-01 00:00:00,粤BAV0409,114.064263,22.518955,2023-02-28 22:19:07,NaN,5.0,96.0,256,0,NaN,NaN,NaN,NaN,119166.2,0.0,0
1,2023-03-01 00:00:00,粤BDR4746,114.137750,22.570602,2023-02-28 22:03:05,NaN,28.0,148.0,768,268566528,NaN,NaN,NaN,NaN,417048.0,0.0,45
2,2023-03-01 00:00:01,粤BD97957,114.129683,22.551567,2023-02-28 22:15:28,NaN,0.0,28.0,768,268435456,NaN,NaN,NaN,NaN,674710.0,0.0,81
3,2023-03-01 00:00:01,粤BDF7305,114.142747,22.555637,2023-02-28 22:23:36,NaN,53.0,94.0,256,268435456,NaN,NaN,NaN,NaN,602294.0,0.0,70
4,2023-03-01 00:00:01,粤BAN1395,114.120495,22.547132,2023-02-28 22:19:39,NaN,55.0,268.0,768,131072,NaN,NaN,NaN,NaN,95.1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220675386,2023-03-01 23:59:57,粤BDK1409,114.269750,22.768173,2023-03-01 22:26:15,NaN,34.0,232.0,256,131072,NaN,NaN,NaN,NaN,116020.1,0.0,0
220675387,2023-03-01 23:59:57,粤BD87070,114.020517,22.531595,2023-03-01 22:39:13,NaN,4.0,144.0,256,268435456,NaN,NaN,NaN,NaN,553679.0,0.0,58
220675388,2023-03-01 23:59:57,粤BDG9356,114.122922,22.541225,2023-03-01 22:33:45,NaN,12.0,304.0,768,268435456,NaN,NaN,NaN,NaN,556536.0,0.0,84
220675389,2023-03-01 23:59:57,粤BD04268,113.811762,22.627985,2023-03-01 22:19:26,NaN,0.0,296.0,256,268435456,NaN,NaN,NaN,NaN,663896.0,0.0,65


In [11]:
#数据清洗
df = df.drop(['ptime', 'no_1','no_2','no_3','no_4','no_5','no_6','no_7','cardno', 'status'], axis=1) #删除不需要的列

In [ ]:
#坐标转换
#将taxi的GPS数据由wgs84坐标转换为gcj02坐标下的web墨卡托坐标
from coordinate_transform import wgs_to_gcj02_to_mercator

# #读取文件
# with open("../Datasets/GPS_data/taxi/GPS_taxi.pkl", 'rb') as f:
#     data = pickle.load(f, encoding='bytes')

#data = pd.read_csv("../Datasets/GPS_data/taxi/2018-10-01/part-r-00000", sep=",", header=None)
#data.columns = ['id','lon','lat','time','device','speed','direction','pstatus','warning','cardno','status','color']

transformed_coords = []
for lon, lat in zip(df['lon'].values, df['lat'].values):
    transformed_coords.append(wgs_to_gcj02_to_mercator(lon, lat))
    
df['lon'], df['lat'] = zip(*transformed_coords)
df.info()

: 

In [ ]:
#保存文件
with open("../Datasets/GPS_data/taxi_new/taxi_big1.pkl", 'wb') as f:
    pickle.dump(df, f)

## bus

In [ ]:
#获取bus文件夹下所有csv文件
path = '../Datasets/GPS_data/bus'

file_list = []
for root,dirs,files in os.walk(path): #根目录，文件夹，文件
        for file in files:
                file_list.append(os.path.join(root,file))
print(file_list, '\n', len(file_list))

In [ ]:
#多个csv文件合并为一个pickle文件
dataframes = []
for file in tqdm(file_list):
    df = pd.read_csv(file, sep=",", header=None, encoding='gb18030')
    dataframes.append(df)
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.to_pickle(os.path.join(path,"GPS_bus.pkl"))
print(combined_df.shape)

In [24]:
path = '../Datasets/GPS_data/bus'
with open(os.path.join(path,"GPS_bus_1.0.pkl"), 'rb') as f:
    data_p = pickle.load(f)
data_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278787371 entries, 0 to 278787370
Data columns (total 9 columns):
 #   Column     Dtype  
---  ------     -----  
 0   systime    object 
 1   device     object 
 2   carID      object 
 3   lon        float64
 4   lat        float64
 5   speed      float64
 6   direction  float64
 7   dspeed     float64
 8   mileage    float64
dtypes: float64(6), object(3)
memory usage: 18.7+ GB


In [5]:
#添加列名
header = ['systime','dtype','device','carID','line','subline','company','pstatus','lon','lat','height','ptime','speed','direction','dspeed','mileage','no1','no2','no3','no4','no5']
data_p.columns = header

In [20]:
#数据清洗
#删除不需要的列
data_p = data_p.drop(['dtype','line','subline','company','pstatus','height','ptime','no1','no2','no3','no4','no5'], axis=1)

In [19]:
a = pd.DataFrame(data_p['device'].value_counts())
a

,count
no5,


In [25]:
data_p

,systime,device,carID,lon,lat,speed,direction,dspeed,mileage
0,2020-10-08T23:59:35.000Z,1550756,BS75463D,114.124069,22.686333,0.0,0.0,0.0,24766.000000
1,2020-10-09T00:00:00.000Z,1641152,BS38969D,113.853119,22.817017,0.0,349.0,0.0,118986.000000
2,2020-10-09T00:00:00.000Z,BS02631D,BS02631D,113.849266,22.621254,45.0,0.0,0.0,170635.515625
3,2020-10-09T00:00:00.000Z,BS09345D,BS09345D,113.879250,22.815056,0.0,0.0,0.0,155719.437500
4,2020-10-09T00:00:00.000Z,BS38299D,BS38299D,113.916946,22.543308,46.0,0.0,0.0,191958.687500
...,...,...,...,...,...,...,...,...,...
278787366,2019-10-10T15:41:43.000Z,1549766,BS72475D,114.325363,22.796400,0.0,112.0,0.0,26462.000000
278787367,2019-10-10T15:41:58.000Z,1549766,BS72475D,114.325302,22.796417,0.0,112.0,0.0,26462.000000
278787368,2019-10-10T15:41:58.000Z,1549766,BS72475D,114.325317,22.796400,0.0,112.0,0.0,26462.000000
278787369,2019-10-10T15:42:20.000Z,1549766,BS72475D,114.325317,22.796417,0.0,112.0,0.0,26462.000000


In [26]:
fsize = os.path.getsize(os.path.join(path,'GPS_bus_1.0.pkl')) #获取文件大小,单位为byte
fsize = fsize/float(1024*1024*1024) #单位变成G
fsize

17.730877673253417

In [22]:
#保存文件
with open(os.path.join(path,"GPS_bus_1.0.pkl"), 'wb') as f:
    pickle.dump(data_p, f)

In [34]:
#坐标转换
#将bus的GPS数据由wgs84坐标转换为gcj02坐标下的web墨卡托坐标
from coordinate_transform import wgs_to_gcj02_to_mercator

#读取文件
with open("../Datasets/GPS_data/bus/GPS_bus_1.0.pkl", 'rb') as f:
    data = pickle.load(f, encoding='bytes')

#data = pd.read_csv("../Datasets/GPS_data/taxi/2018-10-01/part-r-00000", sep=",", header=None)
#data.columns = ['id','lon','lat','time','device','speed','direction','pstatus','warning','cardno','status','color']

transformed_coords = []
for lon, lat in zip(data['lon'].values, data['lat'].values):
    transformed_coords.append(wgs_to_gcj02_to_mercator(lon, lat))
    
data['lon'], data['lat'] = zip(*transformed_coords)
data

,systime,device,carID,lon,lat,speed,direction,dspeed,mileage
0,2020-10-08T23:59:35.000Z,1550756,BS75463D,1.270480e+07,2.593807e+06,0.0,0.0,0.0,24766.000000
1,2020-10-09T00:00:00.000Z,1641152,BS38969D,1.267462e+07,2.609549e+06,0.0,349.0,0.0,118986.000000
2,2020-10-09T00:00:00.000Z,BS02631D,BS02631D,1.267419e+07,2.585919e+06,45.0,0.0,0.0,170635.515625
3,2020-10-09T00:00:00.000Z,BS09345D,BS09345D,1.267753e+07,2.609308e+06,0.0,0.0,0.0,155719.437500
4,2020-10-09T00:00:00.000Z,BS38299D,BS38299D,1.268172e+07,2.576515e+06,46.0,0.0,0.0,191958.687500
...,...,...,...,...,...,...,...,...,...
278787366,2019-10-10T15:41:43.000Z,1549766,BS72475D,1.272719e+07,2.607087e+06,0.0,112.0,0.0,26462.000000
278787367,2019-10-10T15:41:58.000Z,1549766,BS72475D,1.272718e+07,2.607089e+06,0.0,112.0,0.0,26462.000000
278787368,2019-10-10T15:41:58.000Z,1549766,BS72475D,1.272718e+07,2.607087e+06,0.0,112.0,0.0,26462.000000
278787369,2019-10-10T15:42:20.000Z,1549766,BS72475D,1.272718e+07,2.607089e+06,0.0,112.0,0.0,26462.000000


In [35]:
#保存文件
with open("../Datasets/GPS_data/bus/GPS_bus_2.0.pkl", 'wb') as f:
    pickle.dump(data, f)